In [ ]:
mode = "dask"  # RESET THE KERNEL if you change this

In [ ]:
#%reload_ext autoreload
#%autoreload 2
import os
# Select the mode in `pandas`, `cudf`, `dask` or `dask-cudf`.
os.environ["VDF_MODE"] = mode  # RESET THE KERNEL if you change this

#import cardif_dask as vdf  # Import Virtual Dataframe
import virtual_dataframe as vdf
import pandas
import numpy as np
import dask
import cudf

print(f"Use {vdf.VDF_MODE.name.upper()}")

# Version `.apply()` avec un seul calcul par ligne
C'est une version ou les paramètres sont répliqués pour chaque ligne, avant le calcul.
Le problème est qu'il faut connaitre d'avance, le nombre de colonne pour les paramètres.

In [ ]:
# Chaque ligne est un ensemble de paramètre pour chaque Model point
model_points=vdf.VDataFrame(
    {
        "id": [1,2,3],
        "data1": [1.0,2.0,3.0],
        "data2": [10.0,20.0,30.0],
    },
    npartitions=2)
model_points = model_points.set_index("id",drop=False)
#model_points.compute()

In [ ]:
# Chaque colonne est un ensemble de paramètre par simulation
all_simulations=vdf.VDataFrame(
    {
        "sim1":[1,2,3],
        "sim2":[4,5,6]
    },
    index=["a","b","c"],
    npartitions=1)

current_simulation = 0

#all_simulations.compute()

In [ ]:
def add_simulations(model_points, all_simulations, current_simulation):
    one_simulation = all_simulations.iloc[:,current_simulation]
    values_simulations=one_simulation.to_backend()
    model_points=model_points.copy()
    for i,c in enumerate(all_simulations.index.to_numpy()):
        model_points[c] = values_simulations[i]
    return model_points

model_points_merged_with_one_simulation = add_simulations(
    model_points,
    all_simulations,
    current_simulation=current_simulation)

#model_points_merged_with_one_simulation.compute()

In [ ]:
def my_kernel(row):
    return (row.data1 + row. data2) * row.a / row.b + row.c
        
r=model_points_merged_with_one_simulation.copy()
r["out"]=model_points_merged_with_one_simulation.apply(my_kernel, axis=1)
r.compute()

# Version `.apply_rows()` avec plusieurs calculs par lignes
C'est une version ou les paramètres sont répliqués pour chaque ligne, avant le calcul.
Le problème est qu'il faut connaitre d'avance, le nombre de colonne pour les paramètres.

In [ ]:
# Chaque ligne est un ensemble de paramètre pour chaque Model point
model_points=vdf.VDataFrame(
    {
        "id": [1,2,3],
        "data1": [1.0,2.0,3.0],
        "data2": [10.0,20.0,30.0],
    },
    npartitions=2)
model_points = model_points.set_index("id",drop=False)
#model_points.compute()

In [ ]:
# Chaque colonne est un ensemble de paramètre par simulation
all_simulations=vdf.VDataFrame(
    {
        "sim1":[1,2,3],
        "sim2":[4,5,6]
    },
    index=["a","b","c"],
    npartitions=1)

current_simulation = 0

#all_simulations.compute()

In [ ]:
def add_simulations(model_points, all_simulations, current_simulation):
    one_simulation = all_simulations.iloc[:,current_simulation]
    values_simulations=one_simulation.to_backend()
    model_points=model_points.copy()
    for i,c in enumerate(all_simulations.index.to_numpy()):
        model_points[c] = values_simulations[i]
    return model_points

model_points_merged_with_one_simulation = add_simulations(
    model_points,
    all_simulations,
    current_simulation=current_simulation)

#model_points_merged_with_one_simulation.compute()
model_points_merged_with_one_simulation

In [ ]:
def my_kernel(data1_s, data2_s, a_s, b_s, c_s, out1, out2):
    for i,(data1,data2,a,b,c) in enumerate(zip(data1_s,data2_s,a_s,b_s,c_s)):
        if data1 > 2000:
            out1[i]=(data1+data2)*a/b+c
        else:
            out1[i]=(data1+data2)
        out2[i]=b+c
        
r=model_points_merged_with_one_simulation.apply_rows(
    my_kernel, 
    incols=
    {
        'data1':'data1_s',
        'data2':'data2_s',
        'a':'a_s',
        'b':'b_s',
        'c':'c_s'
    },
    outcols=
    {
        'out1': np.float64,
        'out2': np.float64,
    },
    kwargs={}
)
#r.compute()

In [ ]:
r

In [ ]:
r.compute()